In [1]:
from opencage.geocoder import OpenCageGeocode
import time
import pandas as pd
import math
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import requests  

API_KEY = "Add your API Key here"  # OpenCage API Key
geocoder = OpenCageGeocode(API_KEY)  # Initializing geocoder
file_path = "Sample addresses.xlsx"
df = pd.read_excel(file_path, header=None)

In [2]:
def get_coordinates(addresses):
    coordinates = []
    for address in addresses:
        endpoint = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={API_KEY}"
        response = requests.get(endpoint)
        payload = response.json()
        if payload["results"]:
            location = payload["results"][0]["geometry"]
            coordinates.append((location["lat"], location["lng"]))
        else:
            print(f"Address not found: {address}")
            coordinates.append((None, None))  # Append None for failed geocoding
        time.sleep(1)  # Respect API rate limits
    return coordinates

def haversine(lat1, long1, lat2, long2):
    R = 6371  # Radius of earth in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(long2 - long1)

    # Haversine formula
    a = (
        math.sin(delta_phi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

def print_solution(manager, routing, solution, coordinates):
    for vehicle_id in range(manager.GetNumberOfVehicles()):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0  # Initialize route_distance here

        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))

        # Add depot to the route
        route.append(manager.IndexToNode(routing.Start(vehicle_id)))

        # Calculate the total distance of the route manually
        for i in range(len(route) - 1):
            lat1, lon1 = coordinates[route[i]]
            lat2, lon2 = coordinates[route[i + 1]]
            route_distance += haversine(lat1, lon1, lat2, lon2)

        print(f"Route for vehicle {vehicle_id}: {route}")
        print(f"Distance of the route: {route_distance:.2f} km")  # Print total distance in kilometers

def create_data_model():
    addresses = df[0].tolist()  # Gets addresses from the DataFrame
    print("Geocoding addresses...")
    coordinates = get_coordinates(addresses)  # Gets coordinates for all addresses

    # Filter out None coordinates
    coordinates = [coord for coord in coordinates if coord != (None, None)]
    print(f"Geocoded {len(coordinates)} valid addresses.")
    print("Coordinates:", coordinates)  # Print coordinates

    num_addresses = len(coordinates)
    distance_matrix = np.zeros((num_addresses, num_addresses))

    print("Calculating distance matrix...")
    for i in range(num_addresses):
        for j in range(num_addresses):
            if i != j:
                lat1, lon1 = coordinates[i]
                lat2, lon2 = coordinates[j]
                distance_matrix[i, j] = haversine(lat1, lon1, lat2, lon2)
            else:
                distance_matrix[i, j] = 0

    print("Distance Matrix:\n", distance_matrix)  # Print distance matrix

    data = {
        "distance_matrix": distance_matrix,
        "num_vehicles": 1,  # Just one for TSP
        "depot": 0
    }
    return data, coordinates  

# Initializing data
data, coordinates = create_data_model()

# Routing Index Manager
manager = pywrapcp.RoutingIndexManager(len(coordinates), data["num_vehicles"], data["depot"])
routing = pywrapcp.RoutingModel(manager)

# Transit callback
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    distance = data['distance_matrix'][from_node][to_node]
    return distance

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

Geocoding addresses...
Geocoded 5 valid addresses.
Coordinates: [(40.7484421, -73.9856589), (34.0903921, -118.3879642), (41.878738, -87.6359612), (37.778597, -122.418355), (38.8976997, -77.0365532)]
Calculating distance matrix...
Distance Matrix:
 [[   0.         3947.59443879 1145.73880965 4129.73659468  331.98093117]
 [3947.59443879    0.         2813.37421358  547.47431531 3703.06823465]
 [1145.73880965 2813.37421358    0.         2984.16889503  956.14346369]
 [4129.73659468  547.47431531 2984.16889503    0.         3918.60586507]
 [ 331.98093117 3703.06823465  956.14346369 3918.60586507    0.        ]]


In [4]:
# Solving the problem
print("Solving the routing problem...")
solution = routing.SolveWithParameters(search_parameters)

# Printing solution on console
if solution:
    print_solution(manager, routing, solution, coordinates)
else:
    print("No solution found!")

Solving the routing problem...
Route for vehicle 0: [0, 4, 3, 2, 1, 0]
Distance of the route: 13995.72 km
